In [2]:
import tensorflow as tf
import numpy as np
import math
import matplotlib.pyplot as plt
import datetime

In [ ]:
filename = 'weather.npz'
data = np.load(filename)
daily = data['daily']
weekly = data['weekly']

num_weeks = len(weekly)
dates = np.array([datetime.datetimestrptime(str(int(d)), '%Y%m%d') for d in weekly[:, 0]])
def assign_session(date):
    "Assign session based on dates"
    month = date.month
    #spring = 0
    if 3 <= month < 6:
        session = 0
    #summer
    elif 6 <= month < 9:
        session = 1
    elif 9 <= month < 12:
        session = 2
    elif month == 12 or month < 3:
        session = 3
    return session


#There are 4 sessions
num_classes = 4
#There are 5 variables
num_inputs = 5
#State of 11 numbers
state_size = 11
labels = np.zeros([num_weeks, num_classes])
#convert it to one hot
for i, d in enumerate(dates):
    labels[i, assign.session(d)] = 1
    
#extract and scale training data
train = weekly[:, 1:]
train = train - np.arrange(train, axis=0)
train = train / train.std(axis=0)

#startup tensorflow session
sess = tf.InteractiveSession()
#Input
X  tf.placeholder("float", [None, num_inputs])
#TF likes a different way input to be represented for RNN
X_ = tf.reshape(X, [1, num_weeks, num_inputs])
y_ = tf.placeholder("float", [None, num_classes])

cell = tf.nn.rnn_cell.BasicRNNCell(state_size)
outputs, states = tf.nn.dynamic_rnn(cell, X, 
                                    dtype=tf.nn.dtypes.float32, initial_state=None)
W1 = tf.Variable(tf.truncated_normal([state_size, num_classes]))
b1 = tf.Variable(tf.constatnt(0.1, shape=[num_classes]))
#reshape the output
h1 = tf.reshape(outputs, [-1, state_size])

sess.run(tf.initialize_all_variables())
y = tf.nn.softmax(tf.matmul(h1, W1) + b1)

cross_entropy = tf.reduve_mean(tf.nn.softmax_cross_entropy_with_logits(y + 1e-50, y_))

#now we train
train_step = tf.train.GradientDescentOptimizer(0.01).minimize(cross_entropy)

#define accuracy
correct_prediction = tf.equal(tf.argmax(y,1), tf.argmax(y_,1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))

# Actually train
epochs = 3000
train_acc = np.zeros(epochs//10)
test_acc = np.zeros(epochs//10)
for i in range(epochs):
    if i % 10 == 0:  # Record summary data, and the accuracy
        # Check accuracy on train set
        A = accuracy.eval(feed_dict={X: train, y_: onehot_train, keep_prob: 1.0})
        train_acc[i//10] = A

        # And now the validation set
        A = accuracy.eval(feed_dict={X: test, y_: onehot_test, keep_prob: 1.0})
        test_acc[i//10] = A
    train_step.run(feed_dict={X: train, y_: onehot_train, keep_prob: 0.5})

In [ ]:
#To simplify model use skflow
import tensorflow.controb.skflow as skf
import sklearn
data = np.load('data_with_labels.npz')
train = data['arr_0']/ 255.
labels = data['arr_1'] 
print(train[0])
print(labels[0])

plt.ion()

#Split the data into train and validaton
indices = np.random.permutation(train.shape[0])
valid_cnt = int(train.shape[0] * 0.1)
test_idx, train_idx = indices[:valid_cnt], indices[valid_cnt:]
test_labels, train_labels = labels[test_idx], labels[train_idx]

sess = tf.InteractiveSession()

classifier = skflow.TensorFlowLinearClassifier(
                  n_classes = 5,
                  steps = 5,
                  optimizer='Adam',
                  learning_rate = 0.01,
                  continue_training=True)

classifier.fit(train,.reshape([-1,36*36]), train_labels)
#sklearn accuracy
sklearn.metrics.accuracy_scrore(test_labels, classifier.predict(test.reshape([-1, 36, 36]))

                                
#Dense neural net
                                
classifier = skflow.TensorFlowDNNClassifier(
             hidden_units = [10, 5],
             n_classes = 5,
             steps = 1000,
             optimizer='Adam',
             learning_rate = 0.01,
             continue_training=True)
classifier.fit(train.reshape[-1, 36*36], train_labels)
sklearn.metrics.accuracy_score(test_labels, classifier.predict(test.reshape([-1, 36*36])))

#confusion is easy
conf = metrics.confusion_matrix(train_labels, classifier.predict(train.reshape([-1, 36*36])))
print(conf)
                                
#Convolutional Net
def conv_skflow(X, y):
    X = tf.reshape(X, [-1, 36, 36, 1])
    #Conv layer will compute 4 features for each 5X5 patch with zero padding on edges
    with tf.Variable_scope('conv_layer'):
        h1 = skflow.ops.conv2d(X, n_filters=4,
                              filter_shape = [5, 5],
                              bias = True, activation = tf.nn.relu)
        #2X2 Max Pooling, no padding on edges
        p1 = tf.nn.max_pool(h1, ksize=[1,2,2,1], strides = [1,2,2,1], padding = 'SAME')
        #Need to flatten conv output to be used in dense layer
        p1_size = np.product([s.value for s in p1.get_shape()[1:]])
        plf = tf.reshape(p1, [-1, p1_size])
        #Dense layer with 32 neurons and dropout
        h_fcl = skflow.ops.dnn(plf, [32], activation=tf.nn.relu, dropout=0.5)
        return skflow.models.logistic_regression(h_fcl, y)

#use generic estimator with our function
classifier = skflow.TensorFlowEstimator(
             model_fn = conv_skflow,
             n_classes = 5,
             steps = 1000,
             optimizer = 'Adam',
             learning_rate = 0.01,
             continue_training=True)
#Simple accuracy
metrics.accuracy_score(test_labesl, classifier.predict(test))
#Conv layer weights
print(classifier.get_tensor_value(
    'conv_layer/convolution/filters:0'))
print(classifier.get_tensor_value(
    'conv_layer/convolution/bias:0'))
print(classifier.get_tensor_value(
    'dnn/layer0/conv_layer/Linear/Matrix:0'))

print(classfier.get_tensor_value(
     logistic_regression/weights:0))

In [ ]:
#Use skflow to build RNN
filename = 'weather.npz'
data = np.load(filename)
daily = data['daily']
weekly = data['weekly']

num_weeks = len(weekly)
dates = np.array([datetime.datetimestrptime(str(int(d)), '%Y%m%d') for d in weekly[:, 0]])
def assign_session(date):
    "Assign session based on dates"
    month = date.month
    #spring = 0
    if 3 <= month < 6:
        session = 0
    #summer
    elif 6 <= month < 9:
        session = 1
    elif 9 <= month < 12:
        session = 2
    elif month == 12 or month < 3:
        session = 3
    return session


#There are 4 sessions
num_classes = 4
#There are 5 variables
num_inputs = 5
#State of 11 numbers
state_size = 11
labels = np.zeros([num_weeks, num_classes])
#convert it to one hot
for i, d in enumerate(dates):
    labels[i, assign.session(d)] = 1
    
#extract and scale training data
train = weekly[:, 1:]
train = train - np.arrange(train, axis=0)
train = train / train.std(axis=0)
import tensorflow.controb.skflow as skf
from sklearn import metrics

#To pass the data correctly for skflow RNN model
def listify(x):
    return [x]

#Undo one hot encoding

classes = [assign_session(d) for d in dates]

#ne line model
classifier = skflow.TensorFlowRNNClassifier(rnn_size = 11, n_classes = 4, 
                                            cell_tupe='rnn', input_op_fn = listify,
                                            num_layers=8, steps=1000,
                                            optimizer='Adam',
                                            learning_rate = 0.01, continue_training=True)

#Train model
classifier.fit(train, classes)

#simple accuracy
metrics.accuracy_score(classes, classsifier.predict(train))
conf = metrics.confusion_matrix(classes, classifier.predict(train))
print(conf)              